In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

import seaborn as sns
sns.set(color_codes=True)

import pandas as pd

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)


In [ ]:
%ls

In [ ]:
from pandas.io.json import json_normalize
df = pd.read_parquet("results_for_iteration.parquet")
dx = json_normalize(df.results)
dx['load'] = df.load
dx['base_events_per_user'] = df.base_events_per_user
dx['osm_filtered_poi'] = df.osm_filtered_poi
display(dx.head())

dx = dx.groupby(['base_events_per_user', 'load']).count().reset_index()
#dx = dx.groupby(['osm_filtered_poi', 'base_events_per_user', 'load']).count().reset_index()
#display(dx)
dx['load'] = dx['load'].astype(int)
dx = dx.drop(['timing'], axis=1)
#dx.columns = ['load', 'count']
dx.sort_values(['load']).reset_index(drop=True)

In [ ]:
from pandas.io.json import json_normalize
df = pd.read_parquet("results_for_iteration.parquet")
dx = json_normalize(df.results)
dx['load'] = df.load
dx['base_events_per_user'] = df.base_events_per_user
dx['osm_filtered_poi'] = df.osm_filtered_poi
df= dx
df['results__key'] = df['key']
df['results__timing'] = df['timing']
df = df.drop(['key', 'timing'], axis=1)


df.load = df.load.astype(int)
#display(df['load'].value_counts().reset_index().sort_values(['index']))
df = df.sort_values(by=['load'], ascending=True)
df = df.reset_index(drop=True)
df['load (users)'] = df.load
df['time in secons (avg of 5 runs each)'] = df.results__timing
df['variant'] = df.results__key
df = df.drop(['load', 'results__timing', 'results__key'], axis=1)
df.head()

In [ ]:
print(df.variant.unique())
df.loc[df.variant == 'LocalityPreserving', 'variant'] = '(3) LocalityPreserving'
df.loc[df.variant == 'DistributedAlreadyExplodedNoLocalityPreservedOnlyInnerJoin', 'variant'] = '(1) DistributedAlreadyExplodedNoLocalityPreservedOnlyInnerJoin'
df.loc[df.variant == 'DistributedWithExplode', 'variant'] = '(2) DistributedWithExplode'
print(df.variant.unique())

df = df.sort_values(['variant'])
df["log1p(time) in secons (avg of 5 runs each)"] = np.log1p(df["time in secons (avg of 5 runs each)"])

df.to_csv('cleaned.csv', index=False)

In [ ]:
# df = df[df['load (users)'] < 2048]

In [ ]:
#events_per_user = 2000
osm_filtered_poi = 'True'
#osm_filtered_poi = 'False'
events_per_user = 200
df_f = df[(df.base_events_per_user == events_per_user) & (df.osm_filtered_poi== osm_filtered_poi)]
df_f['load (users)'].value_counts()

fig_space = 0.16

In [ ]:
sns.set(font_scale=1.7)

In [ ]:
a4_dims = (15, 8)
fig, ax = pyplot.subplots(figsize=a4_dims)
g = sns.barplot(x="load (users)", y="time in secons (avg of 5 runs each)", hue="variant", data=df_f)
g.set_xticklabels(g.get_xticklabels(), rotation=45)
plt.gcf().subplots_adjust(bottom=fig_space)
g.get_figure().savefig(f"../publication/images/bar_regular_{events_per_user}_{osm_filtered_poi}.png")

In [ ]:
a4_dims = (15, 8)
fig, ax = pyplot.subplots(figsize=a4_dims)
g = sns.barplot(x="load (users)", y="log1p(time) in secons (avg of 5 runs each)", hue="variant", data=df_f)
g.set_xticklabels(g.get_xticklabels(), rotation=45)
plt.gcf().subplots_adjust(bottom=fig_space)
g.get_figure().savefig(f"../publication/images/bar_log_{events_per_user}_{osm_filtered_poi}.png")

In [ ]:
sns.set_style(style='white')
a4_dims = (15, 8)
fig, ax = pyplot.subplots(figsize=a4_dims)
g = sns.pointplot(x="load (users)", y="time in secons (avg of 5 runs each)", hue="variant", data=df_f)
g.set_xticklabels(g.get_xticklabels(), rotation=45)
plt.gcf().subplots_adjust(bottom=fig_space)
g.get_figure().savefig(f"../publication/images/line_white_regular_{events_per_user}_{osm_filtered_poi}.png")

In [ ]:
sns.set_style(style='white')
a4_dims = (15, 8)
fig, ax = pyplot.subplots(figsize=a4_dims)
g = sns.pointplot(x="load (users)", y="log1p(time) in secons (avg of 5 runs each)", hue="variant", data=df_f)
g.set_xticklabels(g.get_xticklabels(), rotation=45)
plt.gcf().subplots_adjust(bottom=fig_space)
g.get_figure().savefig(f"../publication/images/line_white_log_{events_per_user}_{osm_filtered_poi}.png")

In [ ]:
df.head()

In [ ]:
d_plot = df[df.osm_filtered_poi == osm_filtered_poi]
for v in sorted(d_plot.variant.unique()):
    print(v)
    sns.set_style(style='white')
    a4_dims = (15, 8)
    fig, ax = pyplot.subplots(figsize=a4_dims)
    g = sns.pointplot(x="load (users)", y="log1p(time) in secons (avg of 5 runs each)", hue='base_events_per_user',
                      #markers=["o", "x", 'd'], linestyles=["-", "--", '---'],
                      data=d_plot)
    g.set_xticklabels(g.get_xticklabels(), rotation=45)
    plt.title(v)
    plt.gcf().subplots_adjust(bottom=fig_space)
    g.get_figure().savefig(f"../publication/images/run_comparison_{v}_{osm_filtered_poi}.png")